In [1]:
import os
import shutil
from pathlib import Path
from google.colab import files

# ==========================================
# 1. SETUP DIRECTORIES
# ==========================================
demo_path = Path("demos/aircraft_identifier")
if demo_path.exists():
    shutil.rmtree(demo_path)
(demo_path / "examples").mkdir(parents=True, exist_ok=True)
print(f"[1/6] Created directory: {demo_path}")


[1/6] Created directory: demos/aircraft_identifier


In [7]:
# ==========================================
# 2. GENERATE CLASS NAMES
# ==========================================
import os
from pathlib import Path

# 1. Search for the dataset in probable Drive locations
possible_paths = [
    "/content/drive/MyDrive/fgvc-aircraft-2013b",
]

data_path = None
for p in possible_paths:
    if os.path.exists(p):
        data_path = Path(p)
        print(f"✅ Found dataset at: {data_path}")
        break

class_names = []
if data_path:
    # 2. Extract class names (folders only)
    class_names = sorted([entry.name for entry in data_path.iterdir() if entry.is_dir()])
else:
    print("⚠️ WARNING: Could not find dataset in Drive. Checking local environment...")
    # Fallback: Check if you copied it locally during a previous step
    local_data = Path("data/fgvc-aircraft-2013b/train")
    if local_data.exists():
        class_names = sorted([entry.name for entry in local_data.iterdir() if entry.is_dir()])
    else:
        print("❌ CRITICAL: Could not find dataset anywhere. Using dummy list.")
        class_names = ["Plane A", "Plane B", "Plane C"]

# 3. Save to file
with open(demo_path / "class_names.txt", "w") as f:
    f.write("\n".join(class_names))
print(f"[2/6] Generated class_names.txt with {len(class_names)} classes.")

⚠️ WARNING: Could not find dataset in Drive. Checking local environment...
❌ CRITICAL: Could not find dataset anywhere. Using dummy list.
[2/6] Generated class_names.txt with 3 classes.


In [3]:
# ==========================================
# 3. WRITE SCRIPTS (model.py, app.py, requirements.txt)
# ==========================================
model_code = """import torch
import torchvision
from torch import nn

def create_effnetb2_model(num_classes:int=100, seed:int=42):
    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b0(weights=weights)
    for param in model.features.parameters():
        param.requires_grad = False
    torch.manual_seed(seed)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2, inplace=True),
        nn.Linear(in_features=1280, out_features=num_classes),
    )
    return model, transforms
"""
with open(demo_path / "model.py", "w") as f:
    f.write(model_code)

app_code = """import gradio as gr
import os
import torch
from model import create_effnetb2_model
from timeit import default_timer as timer
from typing import Tuple, Dict

with open("class_names.txt", "r") as f:
    class_names = [name.strip() for name in f.readlines()]

model, transforms = create_effnetb2_model(num_classes=len(class_names))
model.load_state_dict(
    torch.load("aircraft_model.pth", map_location=torch.device("cpu"))
)

def predict(img) -> Tuple[Dict, float]:
    start_time = timer()
    img = transforms(img).unsqueeze(0)
    model.eval()
    with torch.inference_mode():
        pred_probs = torch.softmax(model(img), dim=1)
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    pred_time = round(timer() - start_time, 5)
    return pred_labels_and_probs, pred_time

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Label(num_top_classes=3, label="Top 3 Predictions"), gr.Number(label="Time (s)")],
    examples=[["examples/" + e] for e in os.listdir("examples")],
    title="FGVC Aircraft Identifier ✈️",
    description="Identifies 100 aircraft variants."
)
demo.launch()
"""
with open(demo_path / "app.py", "w") as f:
    f.write(app_code)

with open(demo_path / "requirements.txt", "w") as f:
    f.write("torch>=1.12.0\ntorchvision>=0.13.0\ngradio>=3.1.4")
print("[3/6] Wrote model.py, app.py, and requirements.txt.")

[3/6] Wrote model.py, app.py, and requirements.txt.


In [4]:
# ==========================================
# 4. FIND AND MOVE MODEL
# ==========================================
import shutil

destination = demo_path / "aircraft_model.pth"
found = False

# The training notebook saves the best model as "aircraft_model.pth"
# We search for it in the current directory and subdirectories
target_files = ["aircraft_model.pth", "07_effnetb0_8_epochs.pth"]

print("Searching for trained model...")
for target in target_files:
    for root, dirs, files_in_dir in os.walk("."):
        if target in files_in_dir:
            src = os.path.join(root, target)
            print(f"✅ Found model at: {src}")
            shutil.copy(src, destination)
            print(f"[4/6] Moved model to demo folder.")
            found = True
            break
    if found: break

# Safety Net: If not found, check if it's currently loaded in memory
if not found:
    print("⚠️ Model file not found on disk. Checking memory...")
    try:
        # If you just ran training, 'model' might be in memory
        if 'model' in globals():
            torch.save(model.state_dict(), destination)
            print("✅ Saved 'model' from memory to demo folder.")
            found = True
        else:
            print("❌ ERROR: No model found. Please upload 'aircraft_model.pth' manually.")
            # We create a dummy file just so the zip doesn't fail,
            # but the app won't work correctly without the real weights.
            dummy_model, _ = create_effnetb2_model(num_classes=len(class_names))
            torch.save(dummy_model.state_dict(), destination)
    except Exception as e:
        print(f" -> Error saving model: {e}")

Searching for trained model...
⚠️ Model file not found on disk. Checking memory...
❌ ERROR: No model found. Please upload 'aircraft_model.pth' manually.
 -> Error saving model: name 'create_effnetb2_model' is not defined


In [5]:
# ==========================================
# 5. GET EXAMPLE IMAGE
# ==========================================
try:
    # Use a raw git URL to avoid 429 errors
    url = "https://raw.githubusercontent.com/leandrobmarinho/fgvc-aircraft-classification/master/data/images/0057375.jpg"
    os.system(f"wget {url} -O {demo_path}/examples/0057375.jpg")
except:
    pass
print("[5/6] Downloaded example image.")

[5/6] Downloaded example image.


In [6]:
# ==========================================
# 6. ZIP AND DOWNLOAD
# ==========================================
print("[6/6] Zipping...")
shutil.make_archive("demos/aircraft_app", 'zip', demo_path)

if os.path.exists("demos/aircraft_app.zip"):
    print("\nSUCCESS! Zip file created at: demos/aircraft_app.zip")
    print("Attempting download now...")
    try:
        files.download("demos/aircraft_app.zip")
    except:
        print("Automatic download failed. Check the file browser on the left!")
else:
    print("Error: Zip file was not created.")

[6/6] Zipping...

SUCCESS! Zip file created at: demos/aircraft_app.zip
Attempting download now...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

There were issues with the above code when it came to downloading the correct model. However, my code is not trying to create the best model, but simply practice deployment